# Get score

Of an ADS

After running the evaluate_test_set

In [940]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
from pathlib import Path
from IPython.display import display
import pickle
import math
# from env_utils import read_pso_search_files, detect_convergence
%matplotlib inline

from data_utils import find_match_from_seach_collector, get_unique_search_df, get_time_diff_df

In [941]:
def calculate_adv_score(ttc):
    if ttc == 0:
        collision = 1
    else:
        collision = 0
    if ttc > 10:
        ttc = 10
    A = ttc - collision * 50
    A = (A - -50) / (10 - -50) 
    A = A*100
    return A

def get_single_average_score(exp_path):
    search_df_unique = get_unique_search_df(exp_path)
    search_df = search_df_unique.tail(20)
    scores = []
    for i, row in search_df.iterrows():
        ttc = row["ttc"]
        scores.append(calculate_adv_score(ttc))
    average_score = np.mean(scores)
    return average_score

def K(omega):
    mu = 0.5
    sigma = 0.1
    return math.exp(-((omega - mu) ** 2) / (2 * sigma ** 2))

def get_single_scenario_score(score_w_dict):
    part_a = 0
    part_b = 0
    for w, score in score_w_dict.items():
        part_a += K(w) * score
    part_b = sum([K(w) for w in score_w_dict.keys()])
    
    return part_a/part_b

def get_single_scenario_score_wo_K(score_w_dict):
    part_a = 0
    part_b = 0
    for w, score in score_w_dict.items():
        part_a += score
    part_b = len(score_w_dict)
    
    return part_a/part_b

def get_single_scenario_score_wo_side(score_w_dict):
    # without w=0.0 and 1.0
    part_a = 0
    part_b = 0
    for w, score in score_w_dict.items():
        if w == 0.0 or w == 1.0:
            continue
        part_a += score
    part_b = len(score_w_dict) - 2
    
    return part_a/part_b

def get_single_scenario_score_wo_middle(score_w_dict):
    # without w=0.3 ,0.5 and 0.7
    part_a = 0
    part_b = 0
    for w, score in score_w_dict.items():
        if 0.2 < w < 0.8:
            continue
        part_a += score
    part_b = len(score_w_dict) - 3
    
    return part_a/part_b

def get_single_scenario_score_only_high(score_w_dict):
    # 0.7, 1.0
    part_a = 0
    part_b = 0
    for w, score in score_w_dict.items():
        if w < 0.7:
            continue
        part_a += score
    part_b = 2
    
    return part_a/part_b

def get_single_scenario_score_only_high(score_w_dict):
    # 0.7, 1.0
    part_a = 0
    part_b = 0
    for w, score in score_w_dict.items():
        if w < 0.7:
            continue
        part_a += score
    part_b = 2
    
    return part_a/part_b

def get_single_scenario_score_only_high_K(score_w_dict):
    # 0.7, 1.0
    ws = [0.7, 1.0]
    part_a = 0
    part_b = 0
    for w, score in score_w_dict.items():
        if w not in ws:
            continue
        part_a +=  K(w) * score
    part_b = sum([K(w) for w in ws])
    
    return part_a/part_b

def get_single_scenario_score_only_low(score_w_dict):
    # 0.0, 0.3
    part_a = 0
    part_b = 0
    for w, score in score_w_dict.items():
        if w > 0.3:
            continue
        part_a += score
    part_b = 2
    
    return part_a/part_b

def get_single_scenario_score_only_low_K(score_w_dict):
    # 0.0, 0.3
    ws = [0.0, 0.3]
    part_a = 0
    part_b = 0
    for w, score in score_w_dict.items():
        if w not in ws:
            continue
        part_a += K(w)*score
    part_b = sum([K(w) for w in ws])
    
    return part_a/part_b

def get_single_scenario_score_only_0(score_w_dict):
    # 0.0
    part_a = 0
    part_b = 0
    for w, score in score_w_dict.items():
        if w != 0.0:
            continue
        part_a += score
    part_b = 1
    
    return part_a/part_b

def get_single_scenario_score_only_half(score_w_dict):
    # 0.5
    part_a = 0
    part_b = 0
    for w, score in score_w_dict.items():
        if w != 0.5:
            continue
        part_a += score
    part_b = 1
    
    return part_a/part_b

def get_single_scenario_score_only_1(score_w_dict):
    # 1.0
    part_a = 0
    part_b = 0
    for w, score in score_w_dict.items():
        if w != 1.0:
            continue
        part_a += score
    part_b = 1
    
    return part_a/part_b

In [942]:
def make_score(score_fn, score_dict):
    # score for each scenario
    scenario_score_dict = {}

    for scenario_name, w_dict in score_dict.items():
        scenario_score_dict[scenario_name] = score_fn(w_dict)

    scenario_name_list = ["front_brake", "front_cut_in_with_one_npc", "front_cut_in_with_two_npc",
                        "opposite_vehicle_taking_priority",
                        "nonsignalized_junction_left_turn",
                        "nonsignalized_junction_right_turn"]
    # Show the score for each scenario
    print("===========Average for each scenario:============")
    for scenario_name in scenario_name_list:
        print(scenario_name, round(scenario_score_dict[scenario_name],2))

    # Final score
    final_score_average = np.mean(list(scenario_score_dict.values()))
    final_score_average = round(final_score_average, 2)
    print("=======================")
    print(f"Overall average:{final_score_average}")
    return final_score_average

In [943]:
def get_agent_score(agent_name, base_path):
    score_dict = {}

    ADS_name = agent_name

    # Traverse the output directory
    for scenario_name in os.listdir(base_path):
        score_dict[scenario_name] = {}
        # Check if it is a directory
        scenario_path = os.path.join(base_path, scenario_name)
        if os.path.isdir(scenario_path):
            # Traverse the agent directory under each scenario
            HAS_AGENT_DIR = False
            for dir_name in os.listdir(scenario_path):
                if ADS_name == dir_name:
                    HAS_AGENT_DIR = True
                    break

            if HAS_AGENT_DIR:
                for agent_name in os.listdir(scenario_path):
                    if ADS_name == agent_name:
                        agent_path = os.path.join(scenario_path, agent_name)
                        scenario_path = agent_path
                        break

            for experiment_name in os.listdir(scenario_path):

                w = float(experiment_name.split('_')[1])

                score_dict[scenario_name][w] = 0

                exp_path = os.path.join(scenario_path, experiment_name)

                average_score = get_single_average_score(Path(exp_path))

                score_dict[scenario_name][w] = average_score   

    # # Show the second layer structure of the dict
    for k, v in score_dict.items():
        print(k)
        for k2, v2 in v.items():
            print(k2, v2)
        print()
                
    cols = ['ADS','Purposed Method',	'w/o K',	'w/o 0.0,1.0',	'w/o 0.3-0.7',	'High Risks Only',
            'Low Risks Only',	'0.0 only',	'0.5 only',	'1.0 only',	
            'High Risks Only K',	'Low Risks Only K']
    
    scores_by_method = [ADS_name]
    print("\n>>>>>>>>>>Original Method<<<<<<<<<<")
    scores_by_method.append(make_score(get_single_scenario_score, score_dict))
    # scores_by_method.append(make_score(get_single_scenario_score_wo_K, score_dict))
    # scores_by_method.append(make_score(get_single_scenario_score_wo_side, score_dict))
    # scores_by_method.append(make_score(get_single_scenario_score_wo_middle, score_dict))
    # scores_by_method.append(make_score(get_single_scenario_score_only_high, score_dict))
    # scores_by_method.append(make_score(get_single_scenario_score_only_low, score_dict))
    print("\n>>>>>>>>>>Low Risk Only<<<<<<<<<<")
    scores_by_method.append(make_score(get_single_scenario_score_only_0, score_dict))
    # scores_by_method.append(make_score(get_single_scenario_score_only_half, score_dict))
    print("\n>>>>>>>>>>High Risk Only<<<<<<<<<<")
    scores_by_method.append(make_score(get_single_scenario_score_only_1, score_dict))
    # scores_by_method.append(make_score(get_single_scenario_score_only_high_K, score_dict))
    # scores_by_method.append(make_score(get_single_scenario_score_only_low_K, score_dict))

    return scores_by_method

In [944]:
# s_highway = get_agent_score("highway", Path("../../output-final"))
# s_ba = get_agent_score("ba", Path("../../output-final"))
# s_interfuser = get_agent_score("interfuser", Path("../../output-final"))
# s_tfpp = get_agent_score("tfpp", Path("../../output-final"))
# s_apollo = get_agent_score("apollo", Path("../../output-eva-apollo-9-uc39"))
# s_apollo_8 = get_agent_score("apollo", Path("../../output-eva-apollo-8"))
s_apollo_7 = get_agent_score("apollo", Path("../../output-eva-apollo-7"))

cols = ['ADS','Purposed Method',	'w/o K',	'w/o 0.0,1.0',	'w/o 0.3-0.7',	'High Risks Only',
        'Low Risks Only',	'0.0 only',	'0.5 only',	'1.0 only',	
        'High Risks Only K',	'Low Risks Only K']

all_df = pd.DataFrame([s_highway, s_ba, s_interfuser, s_tfpp, s_apollo, s_apollo_8, s_apollo_7], columns=cols)
# drop index
all_df = all_df.reset_index(drop=True)
display(all_df)
# save csv
all_df.to_csv("score_by_type.csv", index=False)

nonsignalized_junction_left_turn
0.7 12.855
0.3 12.664166666666667
1.0 25.525
0.5 83.9075
0.0 81.37249999999999

front_cut_in_with_two_npc
0.7 93.40083333333334
0.0 70.0
0.5 99.2475
0.3 90.29083333333334
1.0 78.45083333333335

opposite_vehicle_taking_priority
0.5 85.45833333333334
0.7 64.48666666666666
0.3 90.88416666666666
1.0 88.705
0.0 97.05249999999998

front_cut_in_with_one_npc
0.3 87.74083333333333
0.0 91.51666666666667
1.0 96.955
0.5 58.388333333333335
0.7 99.0725

front_brake
1.0 75.74249999999999
0.7 71.43666666666665
0.3 35.839999999999996
0.0 83.18583333333333
0.5 88.25083333333333

nonsignalized_junction_right_turn
1.0 60.23166666666666
0.5 51.70166666666667
0.3 30.479999999999997
0.0 40.44833333333333
0.7 60.16166666666667


>>>>>>>>>>Original Method<<<<<<<<<<
===========Average for each scenario:============
front_brake 80.88
front_cut_in_with_one_npc 65.85
front_cut_in_with_two_npc 97.67
opposite_vehicle_taking_priority 83.8
nonsignalized_junction_left_turn 68.75
nonsign

ValueError: 12 columns passed, passed data had 4 columns